In [ ]:
"""
Install required packages if not already installed.

crewai: A framework for orchestrating AI agents & tasks. Enables you to define CrewAI agents, tools, and run workflows.
pyowm: Python client library for OpenWeatherMap API. Provides a wrapper for OpenWeather API, but you are using OpenWeatherMapAPIWrapper 
instead.
openai: OpenAI’s official Python client for GPT models.	Allows GPT-4o to analyze weather data and generate natural language forecasts.
boto3: AWS SDK for Python, used to interact with S3, Lambda, DynamoDB, etc.	Required if you want to store weather reports in AWS S3 or 
automate execution with Lambda.
langchain_community: New module in LangChain that contains community-maintained tools, utilities, and 
integrations. 
"""

!pip install crewai langchain_community pyowm openai boto3 

In [ ]:
!pip install --upgrade pyowm

In [ ]:
!pip show pyowm

In [ ]:
# Import necessary libraries
import os
import json
from crewai import Agent, Task, Crew  # CrewAI core functionality
from datetime import datetime
import boto3 # to create the s3 client

# Importing necessary tools and utilities
from crewai.tools import BaseTool  # For defining custom tools in CrewAI
from pydantic import Field  # For defining tool attributes
from langchain_community.utilities import OpenWeatherMapAPIWrapper  # OpenWeather API wrapper

In [ ]:
# Set API Keys securely using environment variables
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY" 
os.environ["OPENWEATHERMAP_API_KEY"] = "YOUR_WEATHERMAP_API_KEY" 
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o"  

In [ ]:
# Manually configure AWS IAM credentials
os.environ["AWS_ACCESS_KEY_ID"] = "YOUR_IAM_ACCESS_KEY"  
os.environ["AWS_SECRET_ACCESS_KEY"] = "YOUR_IAM_SECRET_KEY" 
os.environ["AWS_REGION"] = "us-east-1"  # Default aws region

In [ ]:
# Set AWS S3 Configuration
S3_BUCKET = "YOUR_BUCKET_NAME"

In [ ]:
# Initialize S3 Client
s3_client = boto3.client("s3")

# Function to Save Data to S3
def save_to_s3(file_name, text_data):
    """Uploads CrewAI results (forecast) to AWS S3"""

    # Convert CrewAI Output to String
    if not isinstance(text_data, str):
        text_data = str(text_data)  # Convert CrewAI object to string
    
    file_path = f"weather_reports/{file_name}"  # Organizing reports in S3 folder
    s3_client.put_object(Bucket=S3_BUCKET, Key=file_path, Body=text_data)
    print(f"Report saved to S3: s3://{S3_BUCKET}/{file_path}")

In [ ]:
# Create an instance of OpenWeather API Wrapper
weather = OpenWeatherMapAPIWrapper()

# Define a Custom Weather Tool for CrewAI
class WeatherTool(BaseTool):
    """
    A custom CrewAI tool to fetch real-time weather information 
    from OpenWeather API.
    """
    name: str = "Weather"
    description: str = "Weather tool for fetching real-time weather information for a given location."
    
    # Using OpenWeather API Wrapper to fetch data
    weather: OpenWeatherMapAPIWrapper = Field(default_factory=OpenWeatherMapAPIWrapper)

    def _run(self, location: str) -> str:
        """
        Fetches weather data for a given location using OpenWeatherMap API.
        Returns JSON data containing temperature, humidity, and weather conditions.
        """
        try:
            return self.weather.run(location)
        except Exception as e:
            return f"Error getting weather: {str(e)}"

In [ ]:
# Instantiate the Weather Tool
weather_tool = WeatherTool()

In [ ]:
# Define the DataFetcher Agent
data_fetcher = Agent(
    name="DataFetcher",
    role="Weather Data Collector",
    goal="Retrieve real-time weather for {location}.",
    backstory="An AI meteorologist specializing in structured weather data collection.",
    tools=[weather_tool]  # Attaching the weather tool to this agent
)

# Define the DataAnalyzer Agent (Using GPT-4o)
data_analyzer = Agent(
    name="DataAnalyzer",
    role="Weather Data Analyst",
    goal="Analyze weather data and generate a detailed forecast (one paragraph max). "
         "Also, the given temperature values are in Celsius. Please convert to Fahrenheit. "
         "Use the formula: °F = °C × (9/5) + 32",
    backstory="An AI-driven meteorologist specializing in natural language weather reports."
)

In [ ]:
# Define CrewAI Tasks
task1 = Task(
    description="Fetch real-time weather data for {location} using OpenWeather API.",
    agent=data_fetcher,
    expected_output="Wind speed, humidity, temperature (in C), rain, heat index, and cloud cover."
)

task2 = Task(
    description="Analyze the following weather data and generate a professional forecast.",
    agent=data_analyzer,
    expected_output="A human-readable weather report summarizing conditions.",
    input=task1  # Task 2 receives weather info from Task 1
)

In [ ]:
# Define CrewAI Workflow
crew = Crew(
    agents=[data_fetcher, data_analyzer],
    tasks=[task1, task2],
    verbose=True  # Enable verbose mode to see CrewAI execution logs
)

#  Before Running CrewAI, identify target city (could get this from user input or some other source)
user_city = "Philadelphia"

# Execute CrewAI Workflow with city
results = crew.kickoff(inputs={"location": user_city})

# Save Weather Report to S3
report_data = results
file_name = f"{user_city}_weather_report_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.txt"
save_to_s3(file_name, report_data)